## Squeezing the most out of data

- Making data useful before training a model
- Representing data in forms that help models learn
- Increasing predictive quality
- Reducing dimensionality with feature engineering

-> benefit: reduce compute resources required, so as cost

**Feature engineering during training must be also applied correctly during serving**

## Main preprocessing operations

* **Data cleansing**: eliminating or correcting erroneous data

* **Feature tuning**: scaling or normalizing 

* **Dimensionality reduction**: reducing the number of features by creating lower dimension and more robust data represents

* **Feature construction**: create new features by using several different techniques

## Empirical knowledge of data

* **Text**: stemming, lemmatization, TF-IDF, n-grams, embedding lookup

* **Image**: clipping, resizing, cropping, blur, Canny filters, Sobel filters, photometric distortions

## Feature engineering technique

Numerical range:

1. Feature scaling
* Grayscale imge pixel intensity scale $[0, 255]$ usually rescaled to  $[-1,1]$ (image = (image - 127.5)/127.5)

2. Normalization and standardization
* Normalization $X_{norm} = \frac{X - X_{min}}{X_{max}  -  X_{min}}$, $X_{norm}\in[0,1]$
* Standardization (y: Z-score), centered on zero <br>
(try both during testing)

Grouping:

3. Bucketizing / Binning
* One-hot encoding for data ranges like histogram ( binning with Facets)

4. Other techniques

Dimensionality reduction:
* PCA: project data along the principal components
* t-SNE: t-Distributed stochastic neighbor embedding
* UMAP: uniform manifold approximation and projection

Feature crossing:
* combine multiple features together into a new feature
* encode nonlinearity in the feature space, or encode same information in fewer features<br>
(e.g. [Day of week, Hour] => [Hour of week])

## TensorFlow embedding projector

* Intuitive exploration of high-dimensional data

<img src="../Figs/TF_projector.png" width="800"/>

Link: projector.tensorflow.org

## tf.Transform

<img src="../Figs/tfTransform.png" width="800"/>

**Transform graph**: the graph expresses all of the transformations on data

<img src="../Figs/tfTransform2.png" width="800"/>

## tf.Transform Analyzers

Analyzer full pass over the data to find constants such as min and max.

<img src="../Figs/tfana.png" width="700"/>

Beam DirectRunner for single system

In [1]:
import tensorflow as tf
import apache_beam as beam
import apache_beam.io.iobase

import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam


In [2]:
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

import pprint
import tempfile

print(f'TensorFlow version: {tf.__version__}')
print(f'TFX Transform version: {tft.__version__}')

TensorFlow version: 2.6.0
TFX Transform version: 1.3.0


## Collect raw data

## Define the metadata

This contains the schema that tells the types of each feature column (or key) in `raw_data`. You need to take note of a few things:

* The transform function later expects the metadata to be packed in a [DatasetMetadata](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/tf_metadata/dataset_metadata.py#L23) object. 
* The constructor for the `DatasetMetadata` class expects a [Schema protocol buffer](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto#L46) data type. You can use the [schema_from_feature_spec()](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/tf_metadata/schema_utils.py#L36) method to generate that from a dictionary.
* To build the said dictionary, you will use the keys/column names of `raw_data` and assign a [FeatureSpecType](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/common_types.py#L29) as values. This allows you to specify if the input is fixed or variable length (using [tf.io](https://www.tensorflow.org/api_docs/python/tf/io) classes), as well as to define the shape and data type.


In [ ]:
# define the schema as a DatasetMetadata object
raw_data_metadata = dataset_metadata.DatasetMetadata(
    
    # use convenience function to build a Schema protobuf
    schema_utils.schema_from_feature_spec({
        
        # define a dictionary mapping the keys to its feature spec type
        'y': tf.io.FixedLenFeature([], tf.float32),
        'x': tf.io.FixedLenFeature([], tf.float32),
        's': tf.io.FixedLenFeature([], tf.string),
    }))

# preview the schema
print(raw_data_metadata._schema)

## Create a preprocessing function
The _preprocessing function_ is the most important concept of `tf.Transform`. A preprocessing function is where the transformation of the dataset really happens. It accepts and returns a dictionary of tensors, where a tensor means a <a target='_blank' href='https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Tensor'><code>Tensor</code></a> or <a target='_blank' href='https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/SparseTensor'><code>SparseTensor</code></a>. There are two main groups of API calls that typically form the heart of a preprocessing function:

1. **TensorFlow Ops:** Any function that accepts and returns tensors. These add TensorFlow operations to the graph that transforms raw data into transformed data one feature vector at a time.  These will run for every example, during both training and serving.
2. **TensorFlow Transform Analyzers:** Any of the analyzers provided by `tf.Transform`. Analyzers also accept and return tensors, but unlike TensorFlow ops they only run once during training, and typically make a full pass over the entire training dataset. They create <a target='_blank' href='https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/constant'>tensor constants</a>, which are added to your graph. For example, `tft.min` computes the minimum of a tensor over the training dataset.

*Caution: When you apply your preprocessing function to serving inferences, the constants that were created by analyzers during training do not change.  If your data has trend or seasonality components, plan accordingly.*

You can see available functions to transform your data [here](https://www.tensorflow.org/tfx/transform/api_docs/python/tft).

In [ ]:
def preprocessing_fn(inputs):
    """Preprocess input columns into transformed columns."""
    
    # extract the columns and assign to local variables
    x = inputs['x']
    y = inputs['y']
    s = inputs['s']
    
    # data transformations using tft functions
    x_centered = x - tft.mean(x)
    y_normalized = tft.scale_to_0_1(y)
    s_integerized = tft.compute_and_apply_vocabulary(s)
    x_centered_times_y_normalized = (x_centered * y_normalized)
    
    # return the transformed data
    return {
        'x_centered': x_centered,
        'y_normalized': y_normalized,
        's_integerized': s_integerized,
        'x_centered_times_y_normalized': x_centered_times_y_normalized,
    }

## Generate a constant graph with the required transformations

Now you're ready to put everything together and transform your data. Like TFDV last week, Tensorflow Transform also uses [Apache Beam](https://beam.apache.org/) for deployment scalability and flexibility. As you'll see below, Beam uses the pipe (`|`) operator to stack the different stages of the pipeline. In this case, you will just feed the data (and metadata) to the [AnalyzeAndTransformDataset](https://www.tensorflow.org/tfx/transform/api_docs/python/tft_beam/AnalyzeAndTransformDataset) class and use the preprocessing function you defined above to transform the data.

For a closer look at Beam syntax for tranform pipelines, you can refer to the documentation [here](https://beam.apache.org/documentation/programming-guide/#applying-transforms).

*Note: You can safely ignore the warning about unparseable args shown after running the cell below.*

# Ignore the warnings
tf.get_logger().setLevel('ERROR')

# a temporary directory is needed when analyzing the data
with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    
    # define the pipeline using Apache Beam syntax
    transformed_dataset, transform_fn = (
        
        # analyze and transform the dataset using the preprocessing function
        (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
            preprocessing_fn)
    )

# unpack the transformed dataset
transformed_data, transformed_metadata = transformed_dataset

# print the results
print('\nRaw data:\n{}\n'.format(pprint.pformat(raw_data)))
print('Transformed data:\n{}'.format(pprint.pformat(transformed_data)))